# Pytorch Lightning Pipeline - Tutorial

We start by importing all the necessary packages.

In [ ]:
import datetime
import os
import time
from pathlib import Path
from typing import Optional

import numpy as np
import pennylane as qml
import torch
from matchcake import NonInteractingFermionicDevice
from matchcake.operations import (
    SingleParticleTransitionMatrixOperation,
)
from torchvision.transforms import Resize

from matchcake_opt.datamodules.datamodule import DataModule
from matchcake_opt.modules.classification_model import ClassificationModel
from matchcake_opt.tr_pipeline.lightning_pipeline import LightningPipeline

Then, we implement our model. In the next cell, you will see an example of classification model that encode images into Ising hamiltonians through a CNN network and then use the Non-Interacting Fermionic (NIF) Device from MatchCake to implement a ansatz that will classify the hamiltonians using a VQE-like optimization.

This model inherit from the `ClassificationModel` that itself inherit from the `BaseModel` which inherit from the `lightning.LightningModule`. So, everything you know or can learn from the lightning module can be apply here to implement your model. The `BaseModel` or `ClassificationModel` are wrapper around the `lightning.LightningModule` used to simplify your life and used to work with the pipelines implemented in this package like the `LightningPipeline` that you will use in this notebook.

Normally, you will want to override the static attribute `MODEL_NAME` for the name of your model. Then you will want to implement the constructor (the `__init__` method) where you will put the instantiation of your network/layer and the objects that you will use later for the computation. Finally, you will implement the `forward` method like any other torch module where you will implement the actual computation/forward pass of your model.

The other methods that you will want to override are all the public methods from the `lightning.LightningModule` itself. See the lightning documentation for that.

In [ ]:
class NIFCNN(ClassificationModel):
    """
    Non-Interacting fermions classifier through CNN hamiltonian encoding. The flow of information
    in the model goes as follows.

    1. CNN(X_{bchw}) -> X_{be}: Build the embeddings with the CNN encoder
    2. Linear(X_{be}) -> W_{bn}: Build the local fields weights from the embeddings
    3. Linear(X_{be}) -> W_{bt}: Build the ZZ couplings weights from the embeddings
    - We now have the hamiltonian: H(X_{be}) = W_{bn} Z_{n} + W_{bt} ZZ_{t}
    4. P_{knu}, P_{ktv}: We compute the probabilities to get the eigenstates of Z and ZZ
    5. E_{k} = W_{bn} P_{knu} Lambda_{u} + W_{bt} P_{ktv} Lambda_{v}: We now compute the expected values with Lambda_{u} and Lambda_{v} the eigenvalues of Z and ZZ respectively.
    6. Softmax(E_{k}) -> Y_{k}: Finally, we apply a softmax to get the probabilities of each class.
    """
    MODEL_NAME = "NIFCNN"
    DEFAULT_N_QUBITS = 16
    DEFAULT_LEARNING_RATE = 2e-4
    DEFAULT_ENCODER_OUTPUT_ACTIVATION = "Tanh"
    MIN_INPUT_SIZE = (28, 28)

    def __init__(
            self,
            input_shape: Optional[tuple[int, ...]],
            output_shape: Optional[tuple[int, ...]],
            learning_rate: float = DEFAULT_LEARNING_RATE,
            n_qubits: int = DEFAULT_N_QUBITS,
            encoder_output_activation: str = DEFAULT_ENCODER_OUTPUT_ACTIVATION,
            **kwargs,
    ):
        super().__init__(input_shape=input_shape, output_shape=output_shape, learning_rate=learning_rate, **kwargs)
        self.save_hyperparameters("learning_rate", "n_qubits", "encoder_output_activation")
        self.n_qubits = n_qubits
        self.R_DTYPE = torch.float32
        self.C_DTYPE = torch.cfloat
        self._n_params = np.triu_indices(2 * self.n_qubits, k=1)[0].size
        self.q_device = NonInteractingFermionicDevice(
            wires=self.n_qubits, r_dtype=self.R_DTYPE, c_dtype=self.C_DTYPE, show_progress=False
        )
        self.encoder_output_activation = encoder_output_activation
        self.input_resize = Resize(self.MIN_INPUT_SIZE)
        self.local_fields_encoder = torch.nn.Sequential(
            torch.nn.LazyConv2d(512, kernel_size=7),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.LazyConv2d(128, kernel_size=5),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.LazyConv2d(64, kernel_size=3),
            torch.nn.LazyBatchNorm2d(),
        )

        self.local_fields_head = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.LazyLinear(self.n_qubits),
            getattr(torch.nn, encoder_output_activation)()
        )
        self._hamiltonian_n_params = np.triu_indices(self.n_qubits, k=1)[0].size
        self.zz_body_couplings_head = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.LazyLinear(self._hamiltonian_n_params),
            getattr(torch.nn, encoder_output_activation)()
        )

        self.zz_body_coupling_weights = torch.nn.Parameter(torch.randn(self._hamiltonian_n_params), requires_grad=True)

        self.local_fields_op_eigvals = torch.nn.Parameter(torch.from_numpy(np.array([1.0, -1.0])).float(),
                                                          requires_grad=False)  # eigvals(Z)
        self.zz_eigvals = torch.nn.Parameter(torch.from_numpy(np.array([1.0, -1.0, -1.0, 1.0])).float(),
                                             requires_grad=False)  # eigvals(ZZ)

        self.local_fields_wires = [[i] for i in range(self.n_qubits)]
        self.couplings_wires = [[i, j] for i, j in np.vstack(np.triu_indices(self.n_qubits, k=1)).T]

        self.weights = torch.nn.Parameter(torch.rand((int(self.output_size), self._n_params)), requires_grad=True)
        torch.nn.init.xavier_uniform_(self.weights)
        self._build()

    def _build(self):
        dummy_input = torch.randn((3, *self.input_shape)).to(device=self.device)
        with torch.no_grad():
            self(dummy_input)
        return self

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.preprocess_input(x)
        embeddings = self.local_fields_encoder(x)

        local_fields = self.local_fields_head(embeddings)
        zz_couplings = self.zz_body_couplings_head(embeddings)
        self.q_device.execute_generator(self.circuit_gen(), reset=True)

        local_fields_probs = (
            self.q_device.probability(wires=self.local_fields_wires)
            .to(dtype=self.q_device.R_DTYPE, device=self.torch_device)
        )
        couplings_probs = (
            self.q_device.probability(wires=self.couplings_wires)
            .to(dtype=self.q_device.R_DTYPE, device=self.torch_device)
        )
        weighted_local_eigvals = torch.einsum(
            "bi,kij,j->bk", local_fields, local_fields_probs, self.local_fields_op_eigvals
        )
        weighted_zz_eigvals = torch.einsum(
            "bi,kij,j->bk", zz_couplings, couplings_probs, self.zz_eigvals
        )

        expval = weighted_local_eigvals + weighted_zz_eigvals
        return expval

    def circuit_gen(self):
        yield qml.BasisState(self.initial_basis_state, wires=self.wires)
        yield self.get_sptm_weights()
        return

    def get_sptm_weights(self):
        """
        Compute the single-particle transition matrix (SPTM) weights based on the initialized weight tensor.

        This method constructs a tensor, `h`, which encodes the pairwise weight interactions
        in a prescribed upper triangular form. It then symmetrizes `h` to ensure anti-symmetry
        about the main diagonal. Afterward, the matrix exponential of `h` is computed to generate
        the SPTM. Finally, the SPTM is encapsulated in a `SingleParticleTransitionMatrixOperation`
        object for further usage.

        :return: An instance of `SingleParticleTransitionMatrixOperation` that encapsulates
            the computed single-particle transition matrix based on the initialized weights.
            The shape of the SPTM is (n_classes, 2 * n_qubits, 2 * n_qubits).
        :rtype: SingleParticleTransitionMatrixOperation
        """
        h = torch.zeros(
            (int(self.output_size), 2 * self.n_qubits, 2 * self.n_qubits), dtype=self.R_DTYPE, device=self.torch_device
        )
        triu_indices = np.triu_indices(2 * self.n_qubits, k=1)
        h[:, triu_indices[0], triu_indices[1]] = self.weights
        h = h - h.mT
        sptm = torch.matrix_exp(h)
        return SingleParticleTransitionMatrixOperation(sptm, wires=self.wires)

    def preprocess_input(self, x: torch.Tensor, **kwargs) -> torch.Tensor:
        x = x.reshape(x.shape[0], -1, x.shape[-2], x.shape[-1])
        if x.shape[-2] < self.MIN_INPUT_SIZE[-2] or x.shape[-1] < self.MIN_INPUT_SIZE[-1]:
            x = self.input_resize(x)
        return x

    @property
    def torch_device(self):
        return self.device

    @property
    def wires(self):
        return self.q_device.wires

    @property
    def initial_basis_state(self):
        return np.zeros(self.n_qubits, dtype=int)

    @property
    def output_size(self):
        return int(np.prod(self.output_shape))


Now, its time to define our training hyperparameters. Since we will use our costume `DataModule` to manipulate the data, we specify the dataset by a string and the other parameters like its presented in the next cell.

In [ ]:
# Dataset
dataset_name = "Digits2D"
fold_id = 0
batch_size = 32
random_state = 0
num_workers = 0

# Model
model_cls = NIFCNN
model_args = dict(
    n_qubits=16,
    learning_rate=2e-4,
    encoder_output_activation="Tanh",
)

# Pipeline
job_output_folder = Path(os.getcwd()) / "data" / "lightning" / dataset_name / model_cls.MODEL_NAME
checkpoint_folder = Path(job_output_folder) / "checkpoints"

It's now time to instantiate the actual datamodule and the training pipeline. We then pass all our predefined hyperparameters.

In [ ]:
datamodule = DataModule.from_dataset_name(
    dataset_name,
    fold_id=fold_id,
    batch_size=batch_size,
    random_state=random_state,
    num_workers=num_workers,
)
lightning_pipeline = LightningPipeline(
    model_cls=model_cls,
    datamodule=datamodule,
    checkpoint_folder=checkpoint_folder,
    max_epochs=10,
    max_time="00:00:03:00",  # DD:HH:MM:SS
    overwrite_fit=True,
    verbose=True,
    **model_args,
)

In the next cell, we will run the training pipline. This one will run the training and validation loop as defined in `lightning`, will save the best model by default and will stop at the maximum time specified.

In [ ]:
start_time = time.perf_counter()
metrics = lightning_pipeline.run()
end_time = time.perf_counter()
elapsed_time = datetime.timedelta(seconds=end_time - start_time)
print(f"Time taken: {elapsed_time}")
print("⚡" * 20, "\nValidation Metrics:\n", metrics, "\n", "⚡" * 20)

Finally, we can test our model on the test set using the best model found during training/validation.

In [ ]:
test_metrics = lightning_pipeline.run_test()
print("⚡" * 20, "\nTest Metrics:\n", test_metrics, "\n", "⚡" * 20)

----------------------------------------